# Solving the travelling salesman problem for the Keyworth Christmas Trail
* Solving the [TSP](https://en.wikipedia.org/wiki/Travelling_salesman_problem) for quickest way to walk around the village [using Google OR-Tools](https://developers.google.com/optimization/routing/tsp)
* See the [Keyworth Community Projects page](https://www.facebook.com/189497114572953/posts/1583160051873312/?extid=0&d=n) if you want to come visit
* Nerdiness by [@Nickopotamus](https://nickopotamus.co.uk)

In [5]:
from __future__ import division
from __future__ import print_function
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import requests
import json
import urllib.request

### Generate distance matrix
* Using [Google Distance Matrix API](https://developers.google.com/optimization/routing/vrp#distance_matrix_api) to build distance matrix

#### Create data model that will store distance matrix
* Had to combine a few nearby locations to make the API work

In [30]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['API_key'] = 'YOUR_KEY_GOES_HERE'
    data['addresses'] = ['18+Ashley+Road+Keyworth+NG12+UK',      #0
                         '54+Ashley+Road+Keyworth+NG12+UK',      #1
                         '46+Beech+Avenue+Keyworth+NG12+UK',     #2
                         '33+Church+Drive+Keyworth+NG12+UK',     #3
                         '1+Covert+Close+Keyworth+NG12+UK',      #4
                         '16+Croft+Road+Keyworth+NG12+UK',       #5
                         '7A+Dale+Road+Keyworth+NG12+UK',        #6
                         '3+East+Close+Keyworth+NG12+UK',        #7
                         '24+Fairway+Keyworth+NG12+UK',          #8
                         '29+Hayes+Road+Keyworth+NG12+UK',       #9
                         '16+Lilac+Close+Keyworth+NG12+UK',      #10
                         '6+Lyncombe+Gardens+Keyworth+NG12+UK',  #11
                         '9+Manor+Road+Keyworth+NG12+UK',        #12
                         '82B+Manor+Road+Keyworth+NG12+UK',      #13 (combined some of Manor Road)
                         'The+Rectory+Nottingham+Road+Keyworth+NG12+UK', #14
                         '32+Park+Avenue+Keyworth+NG12+UK',      #15 (combined some of Park Avenue)
                         '41+Park+Avenue+West+Keyworth+NG12+UK', #16
                         '51+Park+Avenue+West+Keyworth+NG12+UK', #17
                         '2+Rancliffe+Avenue+Keyworth+NG12+UK',  #18
                         '2+Rose+Grove+Keyworth+NG12+UK',        #19
                         '26+Rose+Grove+Keyworth+NG12+UK',       #20
                         '2+Roseland+Close+Keyworth+NG12+UK',    #21
                         '19+Selby+Lane+Keyworth+NG12+UK',       #22 (combined Selby Lane)
                         '13+West+Close+Keyworth+NG12+UK',       #23 (combined West Close)
                         '1+Wolds+Rise+Keyworth+NG12+UK'         #24
                        ]
    data['distance_matrix'] = []
    data['num_vehicles'] = 1
    data['depot'] = 10 # Start at Lilac Close as nearest to the Willowbrook, and furthest from all other points
    return data

#### Compute distance matrix

In [3]:
def create_distance_matrix(data):
  addresses = data["addresses"]
  API_key = data["API_key"]
  max_elements = 100                        # Distance Matrix API only accepts 100 elements per request, so get rows in multiple requests
  num_addresses = len(addresses) 
  max_rows = max_elements // num_addresses  # Maximum number of rows that can be computed per request
  q, r = divmod(num_addresses, max_rows)    # num_addresses = q * max_rows + r
  dest_addresses = addresses
  distance_matrix = []
  
  for i in range(q):                        # Send q requests, returning max_rows rows per request.
    origin_addresses = addresses[i * max_rows: (i + 1) * max_rows]
    response = send_request(origin_addresses, dest_addresses, API_key)
    distance_matrix += build_distance_matrix(response)

  if r > 0:                                  # Get the remaining remaining r rows, if necessary.
    origin_addresses = addresses[q * max_rows: q * max_rows + r]
    response = send_request(origin_addresses, dest_addresses, API_key)
    distance_matrix += build_distance_matrix(response)

  return distance_matrix

#### Build and send API requests

In [4]:
def send_request(origin_addresses, dest_addresses, API_key):
  """ Build and send request for the given origin and destination addresses."""
  def build_address_str(addresses):     # Build a pipe-separated string of addresses
    address_str = ''
    for i in range(len(addresses) - 1):
      address_str += addresses[i] + '|'
    address_str += addresses[-1]
    return address_str

  request = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial'
  origin_address_str = build_address_str(origin_addresses)
  dest_address_str = build_address_str(dest_addresses)
  request = request + '&origins=' + origin_address_str + '&destinations=' + \
                       dest_address_str + '&key=' + API_key
  jsonResult = urllib.request.urlopen(request).read()
  response = json.loads(jsonResult)
  return response

#### Build the distance matrix

In [5]:
def build_distance_matrix(response):
  distance_matrix = []
  for row in response['rows']:
    row_list = [row['elements'][j]['distance']['value'] for j in range(len(row['elements']))]
    distance_matrix.append(row_list)
  return distance_matrix

### Add distance matrix to model

In [6]:
data = create_data_model()
addresses = data['addresses']
API_key = data['API_key']
data['distance_matrix'] = create_distance_matrix(data)
print(data['distance_matrix'])

[[0, 264, 994, 382, 1019, 1000, 482, 846, 701, 883, 1250, 564, 881, 808, 379, 1101, 1209, 1214, 814, 445, 615, 1003, 992, 885, 486], [264, 0, 874, 261, 1069, 1075, 361, 725, 580, 1057, 1129, 813, 760, 952, 258, 981, 1088, 1093, 988, 709, 879, 882, 871, 764, 736], [994, 874, 0, 612, 1143, 1704, 1092, 1354, 587, 1788, 398, 887, 1389, 1581, 989, 1610, 1718, 1722, 1719, 1254, 1424, 1511, 692, 1393, 810], [382, 261, 612, 0, 808, 1193, 479, 843, 319, 1176, 868, 552, 878, 1070, 377, 1099, 1207, 1211, 1106, 1070, 1240, 1000, 665, 882, 475], [819, 1069, 1143, 808, 0, 1565, 1287, 1651, 849, 1449, 1398, 388, 1686, 1374, 1185, 1907, 2015, 2019, 1379, 621, 790, 1808, 1195, 1690, 415], [1195, 1075, 1632, 1193, 1765, 0, 799, 407, 1512, 415, 2195, 1310, 314, 192, 816, 221, 204, 191, 759, 1191, 1361, 618, 1012, 404, 1232], [482, 361, 1092, 479, 1307, 799, 0, 892, 798, 682, 1347, 853, 799, 608, 258, 981, 1088, 1093, 613, 734, 904, 882, 871, 764, 775], [846, 725, 1282, 843, 1889, 407, 725, 0, 1162, 508, 

### Create the routing model

In [7]:
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                       data['num_vehicles'], data['depot'])
routing = pywrapcp.RoutingModel(manager)

### Create the distance callback

In [8]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

### Set the cost of travel

In [9]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
# Set to simply the distance

### Set search parameters
* Using cheapest arc - [but there are other options](https://developers.google.com/optimization/routing/routing_options#first_sol_options)

In [10]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

### Solution printer

In [28]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {}m'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

### Solve and print the solution

In [29]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

Objective: 8716m
Route for vehicle 0:
 10 -> 8 -> 3 -> 24 -> 11 -> 4 -> 20 -> 19 -> 0 -> 1 -> 14 -> 6 -> 18 -> 9 -> 13 -> 5 -> 17 -> 16 -> 15 -> 21 -> 7 -> 23 -> 12 -> 22 -> 2 -> 10



### Save routes to an array for further processing

In [15]:
def get_routes(solution, routing, manager):
  """Get vehicle routes from a solution and store them in an array."""
  # Get vehicle routes and store them in a two dimensional array whose
  # i,j entry is the jth location visited by vehicle i along its route.
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes

### Print order of route

In [27]:
routes = get_routes(solution, routing, manager)

for i in range(len(routes[0])):
    print(str(i) + ": " + data['addresses'][(routes[0][i])])

0: 16+Lilac+Close+Keyworth+NG12+UK
1: 24+Fairway+Keyworth+NG12+UK
2: 33+Church+Drive+Keyworth+NG12+UK
3: 1+Wolds+Rise+Keyworth+NG12+UK
4: 6+Lyncombe+Gardens+Keyworth+NG12+UK
5: 1+Covert+Close+Keyworth+NG12+UK
6: 26+Rose+Grove+Keyworth+NG12+UK
7: 2+Rose+Grove+Keyworth+NG12+UK
8: 18+Ashley+Road+Keyworth+NG12+UK
9: 54+Ashley+Road+Keyworth+NG12+UK
10: The+Rectory+Nottingham+Road+Keyworth+NG12+UK
11: 7A+Dale+Road+Keyworth+NG12+UK
12: 2+Rancliffe+Avenue+Keyworth+NG12+UK
13: 29+Hayes+Road+Keyworth+NG12+UK
14: 82B+Manor+Road+Keyworth+NG12+UK
15: 16+Croft+Road+Keyworth+NG12+UK
16: 51+Park+Avenue+West+Keyworth+NG12+UK
17: 41+Park+Avenue+West+Keyworth+NG12+UK
18: 32+Park+Avenue+Keyworth+NG12+UK
19: 2+Roseland+Close+Keyworth+NG12+UK
20: 9+Manor+Road+Keyworth+NG12+UK
21: 13+West+Close+Keyworth+NG12+UK
22: 3+East+Close+Keyworth+NG12+UK
23: 19+Selby+Lane+Keyworth+NG12+UK
24: 46+Beech+Avenue+Keyworth+NG12+UK
25: 16+Lilac+Close+Keyworth+NG12+UK
